In [25]:
import ast
import json
import os
import re
import sys
import time
import random
import traceback

# Bibliotecas de Terceiros (mais usadas primeiro)
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

# Bibliotecas de Visualização
import matplotlib.pyplot as plt
import seaborn as sns

# Selenium e WebDriver Manager
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

In [14]:
# A função pd.read_csv() é usada para carregar o arquivo CSV (Comma Separated Values)
# para dentro do ambiente de trabalho.

df = pd.read_csv("processos_crimes_fauna_tjce.csv")

# O método .info() é essencial para a Análise Exploratória de Dados (AED).

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   classe                     3000 non-null   object
 1   numeroProcesso             3000 non-null   object
 2   sistema                    3000 non-null   object
 3   formato                    3000 non-null   object
 4   tribunal                   3000 non-null   object
 5   dataHoraUltimaAtualizacao  3000 non-null   object
 6   grau                       3000 non-null   object
 7   @timestamp                 3000 non-null   object
 8   dataAjuizamento            3000 non-null   int64 
 9   movimentos                 3000 non-null   object
 10  id                         3000 non-null   object
 11  nivelSigilo                3000 non-null   int64 
 12  orgaoJulgador              3000 non-null   object
 13  assuntos                   3000 non-null   object
dtypes: int64

In [15]:
# Seleção de Colunas:
# Aqui, criamos um novo DataFrame (df_selecionado) que é um subconjunto do DataFrame original (df).
# Para selecionar colunas específicas, usamos dois pares de colchetes: df[['nome_da_coluna']].

df_selecionado = df [['assuntos']]

# Exibição das Primeiras Linhas:
# O método .head() é fundamental após uma operação de seleção ou filtragem.
# Ele exibe as 5 primeiras linhas do novo DataFrame (df_selecionado).

df_selecionado.head() 

,assuntos
0,"[{'codigo': 3619, 'nome': 'Crimes contra a Fau..."
1,"[{'codigo': 3619, 'nome': 'Crimes contra a Fau..."
2,"[{'codigo': 3619, 'nome': 'Crimes contra a Fau..."
3,"[{'codigo': 3619, 'nome': 'Crimes contra a Fau..."
4,"[{'codigo': 3619, 'nome': 'Crimes contra a Fau..."


In [16]:
# Cria um novo DataFrame contendo apenas a coluna 'numeroProcesso'
# A sintaxe df[['nome_da_coluna']] é usada para selecionar UMA ou MAIS colunas,
# garantindo que o resultado (df_processos) seja um novo DataFrame.
df_processos = df[['numeroProcesso']]

# Exibe as 5 primeiras linhas do novo DataFrame para verificação
print("As primeiras 5 linhas do DataFrame com apenas o numeroProcesso:")
# .head() é uma verificação visual rápida para confirmar que os dados foram selecionados corretamente.
print(df_processos.head())

# Exibe a forma (linhas e colunas) do novo DataFrame para confirmar que é uma única coluna
print("\nForma (Shape) do novo DataFrame:")
# O atributo .shape retorna uma tupla (número_de_linhas, número_de_colunas).
# É usado para confirmar se a seleção resultou no número esperado de colunas (que deve ser 1 neste caso).
print(df_processos.shape)

As primeiras 5 linhas do DataFrame com apenas o numeroProcesso:
         numeroProcesso
0  02038219120258060298
1  02019423420258060303
2  02027894220258060301
3  02017180220258060302
4  00284953820258060001

Forma (Shape) do novo DataFrame:
(3000, 1)


In [17]:
# Garante que 'movimentos' seja lista de dicionários

# DEFINIÇÃO DA FUNÇÃO DE TRATAMENTO:
# Esta função ('tratar_movimentos') é criada para converter a coluna 'movimentos'

def tratar_movimentos(x):
    # 1. Checa se o dado é uma string (o formato mais comum após a leitura do CSV).
    if isinstance(x, str):
        try:
            # ast.literal_eval() é a ferramenta segura para converter uma string
            # que se parece com uma lista ou dicionário em um objeto Python real.
            return ast.literal_eval(x)
        except:
            # Se a conversão falhar (string mal formatada), retorna uma lista vazia.
            return []
    # 2. Se o dado já for uma lista, ele está pronto.
    elif isinstance(x, list):
        return x
    # 3. Para qualquer outro caso (ex: NaN, None), retorna lista vazia.
    else:
        return []

# APLICAÇÃO DA FUNÇÃO:
# .apply() aplica a função 'tratar_movimentos' linha por linha na coluna "movimentos".
# O resultado (agora uma lista real) é salvo na nova coluna "movimentos_tratados".
df["movimentos_tratados"] = df["movimentos"].apply(tratar_movimentos)

# Extrair nomes resumidos dos movimentos
# CRIAÇÃO DE LISTA SIMPLIFICADA:
# Aplica uma função lambda para iterar sobre a lista de dicionários em "movimentos_tratados".
df["lista_movimentos"] = df["movimentos_tratados"].apply(
    # Para cada dicionário 'm' na lista, extrai o valor associado à chave "nome".
    # Se a chave "nome" não existir, retorna uma string vazia ("").
    lambda movs: [m.get("nome", "") for m in movs]
)

# Extrair nome do órgão julgador
# FUNÇÃO PARA EXTRAIR DADOS DE DICIONÁRIOS (Semelhante à anterior):
def extrair_orgao(x):
    # Trata o caso de a coluna "orgaoJulgador" ter sido lida como string.
    if isinstance(x, str):
        try:
            # Converte a string (se for formato de dicionário) em dicionário Python.
            x = ast.literal_eval(x)
        except:
            # Se falhar ou não for string, retorna None.
            return None
    # Se o dado resultante for um dicionário Python (tratado ou já era),
    if isinstance(x, dict):
        # Extrai o valor da chave "nome" do dicionário.
        return x.get("nome")
    # Caso contrário (NaN, None, ou outros formatos), retorna None.
    return None

# APLICAÇÃO DA FUNÇÃO DE EXTRAÇÃO:
# Aplica a função para criar uma nova coluna com o nome limpo do órgão julgador.
df["orgao_julgador_nome"] = df["orgaoJulgador"].apply(extrair_orgao)

# Exibe as primeiras linhas com as colunas recém-criadas para verificar a transformação.

df[["numeroProcesso", "orgao_julgador_nome", "lista_movimentos"]].head()

,numeroProcesso,orgao_julgador_nome,lista_movimentos
0,02038219120258060298,5º NUCLEO REGIONAL DE CUSTODIA E DAS GARANTIAS,"[Conclusão, Distribuição, Documento, Expedição..."
1,02019423420258060303,3º NUCLEO REGIONAL DE CUSTODIA E DAS GARANTIAS,"[Distribuição, Conclusão, Mero expediente]"
2,02027894220258060301,1º NUCLEO REGIONAL DE CUSTODIA E DAS GARANTIAS,"[Conclusão, Distribuição, Conclusão]"
3,02017180220258060302,2º NUCLEO REGIONAL DE CUSTODIA E DAS GARANTIAS,"[Conclusão, Conclusão, Distribuição, Expedição..."
4,00284953820258060001,18ª VARA CRIMINAL DA COMARCA DE FORTALEZA,"[Distribuição, Documento, Documento, Expedição..."


In [18]:
# Verificar se existe algum movimento de sentença

# DEFINIÇÃO DA FUNÇÃO DE CLASSIFICAÇÃO:
# A função 'tem_sentenca' verifica se a lista de movimentos de um processo
# contém algum dos termos definidos que indicam o fim da fase de conhecimento.
def tem_sentenca(lista):
    # Lista de termos-chave a serem procurados. A busca por palavras-chave é a forma mais simples de classificação textual.
    termos = ["sentença", "decisão", "julgado", "concluso para sentença"]
    # 1. Junta todos os movimentos em uma única string (texto).
    # 2. Converte para minúsculas (.lower()) para garantir que a busca não seja sensível a maiúsculas/minúsculas.
    texto = " ".join(lista).lower()
    # Retorna True se QUALQUER (any) dos termos for encontrado (in) dentro do texto.
    return any(t in texto for t in termos)

# APLICAÇÃO DA CLASSIFICAÇÃO:
# Aplica a função 'tem_sentenca' a cada item da coluna "lista_movimentos" (que é uma lista de strings).
# O resultado (True ou False) é armazenado na nova coluna booleana "possui_sentenca_movimento".
df["possui_sentenca_movimento"] = df["lista_movimentos"].apply(tem_sentenca)

# CONTAGEM DE FREQUÊNCIA:
# O método .value_counts() é usado para contar quantas vezes cada valor único
# (True ou False, neste caso) aparece na coluna.
# É usado para ter uma visão geral da proporção de processos que possuem sentença.
df["possui_sentenca_movimento"].value_counts()


possui_sentenca_movimento
True     1848
False    1152
Name: count, dtype: int64

In [19]:
# FILTRAGEM DO DATAFRAME:
# Cria um novo DataFrame (df_sentenca) contendo apenas os processos que
# satisfazem a condição: "possui_sentenca_movimento" igual a True.
# A sintaxe df[...] é usada para filtrar linhas.

df_sentenca = df[df["possui_sentenca_movimento"] == True].copy()

# EXIBIÇÃO DA CONTAGEM:
# len() é uma função Python padrão que retorna o número de itens em um objeto.
# Aqui, ele é usado para contar o número de linhas (processos) no DataFrame filtrado,

print("Total com sentença:", len(df_sentenca))

# CONFERÊNCIA DOS DADOS FILTRADOS:
# Exibe as primeiras 5 linhas das colunas-chave do novo DataFrame filtrado.

df_sentenca[["numeroProcesso", "orgao_julgador_nome", "lista_movimentos"]].head()


Total com sentença: 1848


,numeroProcesso,orgao_julgador_nome,lista_movimentos
15,02033411620258060298,5º NUCLEO REGIONAL DE CUSTODIA E DAS GARANTIAS,"[Conclusão, Expedição de documento, Distribuiç..."
37,00255308720258060001,18ª VARA CRIMINAL DA COMARCA DE FORTALEZA,"[Petição, Conclusão, Expedição de documento, D..."
65,02190163720258060001,18ª VARA CRIMINAL DA COMARCA DE FORTALEZA,"[Conclusão, Expedição de documento, Expedição ..."
85,02019078920258060298,5º NUCLEO REGIONAL DE CUSTODIA E DAS GARANTIAS,"[Expedição de documento, Petição, Conclusão, E..."
112,02928082920228060001,GADES - BENEDITO HELDER AFONSO IBIAPINA,"[Distribuição, Documento, Expedição de documen..."


In [20]:
# Criar URL pública de consulta no TJCE

# CRIAÇÃO DA COLUNA DE URL:
# Utiliza o método .apply() com uma função lambda para construir uma URL única para cada processo.
df_sentenca["url_publica_tjce"] = df_sentenca["numeroProcesso"].apply(
    # Função Lambda:
    # f"..." é uma f-string (string formatada), que permite incorporar o valor da variável 'x'
    # (que é o 'numeroProcesso' de cada linha) diretamente na URL base do TJCE.
    lambda x: f"https://esaj.tjce.jus.br/cpopg/showProcessoDigital.do?processo.numero={x}"
)

# Conferência
# Exibe as colunas "numeroProcesso" e a recém-criada "url_publica_tjce" nas primeiras 5 linhas.
# Isso confirma que a URL foi construída corretamente, substituindo o placeholder do número do processo.
df_sentenca[["numeroProcesso", "url_publica_tjce"]].head()



,numeroProcesso,url_publica_tjce
15,02033411620258060298,https://esaj.tjce.jus.br/cpopg/showProcessoDig...
37,00255308720258060001,https://esaj.tjce.jus.br/cpopg/showProcessoDig...
65,02190163720258060001,https://esaj.tjce.jus.br/cpopg/showProcessoDig...
85,02019078920258060298,https://esaj.tjce.jus.br/cpopg/showProcessoDig...
112,02928082920228060001,https://esaj.tjce.jus.br/cpopg/showProcessoDig...


## Teste para realizar web scraping em um processo

In [35]:
# =======================
# BLOCO 1 - ABRIR PAGINA WEB DO TJCE
# =======================
# CONFIGURAÇÃO DO SELENIUM (SEGURO)
# =======================

# Inicializa o objeto Options para configurar o comportamento do navegador Chrome.
options = Options()
# Define que a janela do navegador deve iniciar maximizada (melhora a visualização e interação).
options.add_argument("--start-maximized")
# Adiciona um argumento para tentar disfarçar que o navegador está sendo controlado por automação.
options.add_argument("--disable-blink-features=AutomationControlled")
# Remove as barras de informação que o Chrome exibe em modo automatizado.
options.add_argument("--disable-infobars")
# Desativa notificações que podem interromper o script.
options.add_argument("--disable-notifications")
# Remove o aviso "Chrome está sendo controlado por software de teste".
options.add_experimental_option("excludeSwitches", ["enable-automation"])
# Desativa extensões de automação.
options.add_experimental_option("useAutomationExtension", False)

# Configura o serviço do ChromeDriver:
# O ChromeDriverManager verifica automaticamente a versão do seu Chrome e baixa o driver compatível.
# Isso elimina a necessidade de gerenciar drivers manualmente, tornando o código mais robusto.
service = Service(ChromeDriverManager().install())
# Inicializa a instância do navegador Chrome, passando as configurações e o serviço do driver.
driver = webdriver.Chrome(service=service, options=options)

# Configurações de Espera Explícita (WebDriverWait):
# Cria um objeto 'wait' que será usado para esperar até 20 segundos
# por um elemento na página antes de lançar uma exceção de Timeout.
wait = WebDriverWait(driver, 20)

# =======================
# ABRIR SITE DO TJCE
# =======================
# Define a URL de destino (página de consulta pública do TJCE).
url = "https://esaj.tjce.jus.br/cpopg/open.do"
# O método .get() instrui o navegador a carregar a URL definida.
driver.get(url)

# Feedback visual para o usuário.
print("Chrome aberto com sucesso.")
print("Página do TJCE carregando...")

# Aguarda 5 segundos. Um 'sleep' simples é usado aqui para dar tempo à página
# para carregar seus elementos iniciais (embora o WebDriverWait seja mais profissional para esperas específicas).
time.sleep(5)

# Confirmação de conclusão do bloco.
print("BLOCO 1 FINALIZADO COM SUCESSO")


Chrome aberto com sucesso.
Página do TJCE carregando...
BLOCO 1 FINALIZADO COM SUCESSO


## Resumo Didático Bloco 1(Foco no Selenium)
Este bloco configura o Motor de Automação (Selenium).

Options() e Argumentos: O coração da configuração do Selenium. Os argumentos como --start-maximized e, principalmente, as opções de excludeSwitches e useAutomationExtension são usados para tornar a navegação do robô mais parecida com a de um humano, o que é crucial para evitar detecção por sites mais robustos.

ChromeDriverManager().install(): Esta linha resolve o maior problema do Selenium: a incompatibilidade de drivers. Ela garante que a versão correta do ChromeDriver seja baixada e usada, independente da versão do Chrome instalada no seu PC.

WebDriverWait: Embora não usado imediatamente, o objeto wait é uma ferramenta avançada. Em vez de usar time.sleep(X) (que sempre espera X segundos, mesmo que a página carregue em 1 segundo), o WebDriverWait espera apenas até a condição ser satisfeita ou o timeout (20s) ser atingido, tornando o scraper mais rápido e confiável.

In [36]:
# =======================
# BLOCO 2 - PREENCHER CAMPOS TJCE CORRETAMENTE
# =======================

# Define o número do processo (CNJ - Conselho Nacional de Justiça) a ser pesquisado.
# O formato CNJ padrão é NNNNNNN-DD.AAAA.J.TR.OOOO.
processo = "02190163720258060001"

# Quebra correta do número CNJ
# Divide a string do número do processo em partes conforme o formato do campo de busca do TJCE.
parte1 = processo[:7]   # NNNNNNN (Primeiros 7 dígitos - Número Sequencial)
parte2 = processo[7:9]  # DD      (Dígitos Verificadores)
parte3 = processo[9:13] # AAAA    (Ano do ajuizamento)
parte4 = processo[-4:]  # OOOO    (Últimos 4 dígitos - Unidade de Origem / Foro)

print("Partes extraídas:")
print(parte1, parte2, parte3, parte4)

# Campo principal: 0000000-00.0000
# USO DO WAITER: A espera explícita é usada para garantir que o elemento (campo de texto)
# com o ID "numeroDigitoAnoUnificado" esteja presente no DOM (Document Object Model) da página.
campo_principal = wait.until(
    EC.presence_of_element_located((By.ID, "numeroDigitoAnoUnificado"))
)
# Limpa qualquer conteúdo pré-existente no campo de texto.
campo_principal.clear()
# Digita a parte formatada (0000000-00.0000) no campo, usando uma f-string para união.
campo_principal.send_keys(f"{parte1}-{parte2}.{parte3}")

# Campo final: 0001
# Espera explícita para garantir que o campo do número do foro (Unidade de Origem) esteja presente.
campo_final = wait.until(
    EC.presence_of_element_located((By.ID, "foroNumeroUnificado"))
)
# Limpa o campo.
campo_final.clear()
# Digita a parte final do número do processo (parte4).
campo_final.send_keys(parte4)

# Pausa simples para que o usuário ou a aplicação veja os campos preenchidos antes de avançar.
time.sleep(2)

print("Campos preenchidos corretamente.")


Partes extraídas:
0219016 37 2025 0001
Campos preenchidos corretamente.


## Resumo Didático (Foco na Interação e Formatação)
Este bloco demonstra três conceitos essenciais:

Formato CNJ: O número do processo segue um padrão nacional. Usar slicing ([:7], [7:9], etc.) é a técnica de Python para quebrar a string e extrair as partes necessárias.

WebDriverWait (wait.until): Este é o método profissional de espera no Selenium. Ele é superior a time.sleep(), pois só espera o tempo estritamente necessário para que o elemento apareça (EC.presence_of_element_located).

Interação com Formulários (.clear() e .send_keys()):

.clear(): Garante que o campo de texto está vazio antes de digitar.

.send_keys(): Simula a digitação de dados pelo usuário.

In [28]:
# =======================
# BLOCO 3 - CLICAR EM CONSULTAR
# =======================

botao_consultar = wait.until(
    EC.element_to_be_clickable((By.ID, "botaoConsultarProcessos"))
)
botao_consultar.click()

print(" Botão Consultar clicado.")

# Aguardar página do processo carregar
time.sleep(6)

print(" Se não houver erro, o processo deve estar aberto na nova tela.")



 Botão Consultar clicado.
 Se não houver erro, o processo deve estar aberto na nova tela.


In [29]:
# =======================
# BLOCO 4: Carregamento Completo das Movimentações (Versão de Produção)
# =======================

# XPATH e configurações do loop

# Define o XPATH (XML Path Language) para localizar o link "Mais" que carrega novas movimentações.
# O XPATH seleciona a tag 'a' (link) que tem o ID 'linkmovimentacoes' E contém o texto 'Mais'.
XPATH_BOTAO_MAIS = "//a[@id='linkmovimentacoes' and contains(., 'Mais')]"
# Define o número máximo de vezes que o robô tentará clicar, como medida de segurança.
TENTATIVAS_MAXIMAS = 15
# Inicializa o contador de tentativas.
tentativas = 0

# Rolar para o final antes de começar
# Usa JavaScript (driver.execute_script) para rolar a página até o final (document.body.scrollHeight).
# Isso garante que o primeiro botão 'Mais' esteja visível, caso ele já esteja lá embaixo.
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
# Pausa simples para a rolagem terminar.
time.sleep(2)

# Loop para clicar no "Mais" até que ele desapareça
# O loop continuará enquanto o número de tentativas for menor que o máximo.
while tentativas < TENTATIVAS_MAXIMAS:
    try:
        # Espera curta pelo link clicável
        # Espera explícita de no máximo 5 segundos (WebDriverWait).
        # EC.element_to_be_clickable garante que o elemento não só existe, mas também pode ser clicado.
        botao_mais = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, XPATH_BOTAO_MAIS))
        )
        
        # Rola o botão para a vista
        # Usa JavaScript para garantir que o botão esteja no centro da tela.
        # Isso é uma técnica de robustez para evitar que outros elementos o cubram.
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", botao_mais)
        time.sleep(1) 
        
        # Clique nativo
        # Executa a ação de clique no elemento encontrado.
        botao_mais.click()
        
        # Aguarda o carregamento
        # Pausa longa para dar tempo ao servidor para buscar e renderizar os novos movimentos na página.
        time.sleep(6) 
        
        # Rola a página para garantir que o novo botão 'Mais' esteja visível
        # Rola novamente para o final, preparando para a próxima iteração do loop, caso haja mais movimentos.
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        
        # Incrementa o contador de sucesso.
        tentativas += 1
        
    except TimeoutException:
        # EXCEÇÃO ESPERADA: Se o WebDriverWait estourar o limite de 5 segundos,
        # significa que o botão 'Mais' não existe mais.
        # Isso indica que todas as movimentações foram carregadas. O loop é interrompido.
        break 
        
    except Exception:
        # Qualquer outro erro, como interceptação inesperada ou erro no clique,
        # trata o processo como concluído (ou irrecuperável) e sai do loop.
        break

print(f" Bloco 4 (Carregamento de Movimentos) Finalizado. {tentativas} cliques realizados.")

 Bloco 4 (Carregamento de Movimentos) Finalizado. 1 cliques realizados.


## Resumo Didático (Foco na Robustez)
Este bloco é um manual de robustez para web scraping dinâmico:

XPATH (XPATH_BOTAO_MAIS): Linguagem de caminho usada para selecionar elementos complexos. O uso de and contains(., 'Mais') torna a seleção precisa e menos propensa a erros de alteração de layout.

driver.execute_script (Manipulação de Tela): É usado para controlar o navegador via JavaScript. Rolar a página é essencial, pois o Selenium só pode interagir com elementos visíveis na tela.

Loop e Tratamento de Exceção (while/try/except): O design do loop é o ponto alto. Em vez de depender de uma condição de sucesso, ele depende de uma condição de falha esperada (TimeoutException). Quando o botão "Mais" não pode mais ser encontrado, o WebDriverWait gera a exceção, o que indica o fim do carregamento e quebra o loop de forma limpa.

TENTATIVAS_MAXIMAS: É uma salvaguarda contra loops infinitos, caso o site esteja quebrado ou o XPATH esteja errado.

In [30]:
# =======================
# BLOCO 5: EXTRAÇÃO FINAL DAS MOVIMENTAÇÕES (TJCE)
# =======================

print("\n# BLOCO 5 - EXTRAÇÃO FINAL DAS MOVIMENTAÇÕES (TJCE)")

try:
    # Tenta localizar diretamente o tbody correto pelo ID
    tbody = driver.find_element(By.ID, "tabelaTodasMovimentacoes")
    print("Encontrado tbody com ID 'tabelaTodasMovimentacoes'")

    linhas = tbody.find_elements(By.CSS_SELECTOR, "tr.containerMovimentacao")
    print(f"total de tr.containerMovimentacao encontradas: {len(linhas)}")

    movimentos = []

    for tr in linhas:
        try:
            data = tr.find_element(By.CSS_SELECTOR, "td.dataMovimentacao").text.strip()
            desc = tr.find_element(By.CSS_SELECTOR, "td.descricaoMovimentacao").text.strip()
            movimentos.append((data, desc))
        except Exception as e:
            print(f"Erro extraindo linha: {e}")
            continue

    print("\n Extração finalizada. registros válidos:", len(movimentos))
    for m in movimentos:
        print(m)

except Exception as e:
    print(f" Erro no bloco 5: {e}")



# BLOCO 5 - EXTRAÇÃO FINAL DAS MOVIMENTAÇÕES (TJCE)
Encontrado tbody com ID 'tabelaTodasMovimentacoes'
total de tr.containerMovimentacao encontradas: 33

 Extração finalizada. registros válidos: 33
('10/12/2025', 'Certidão emitida\nPORTAL - 50235 - Certidão de decurso de prazo (10 dias) para cientificação da intimação eletrônica')
('27/11/2025', 'Expedição de Ofício\n[ÁREA CRIMINAL] - [MALOTE DIGITAL] - Magistrado - Ofício Genérico')
('26/11/2025', 'Certidão emitida\nPORTAL - 50235 - Certidão de remessa da intimação para o Portal Eletrônico')
('18/11/2025', 'Decisão Interlocutória de Mérito\nDiante do exposto, com fundamento no art. 118 do CPP, DEFIRO o requerimento formulado pela autoridade policial e DETERMINO: a) A manutenção da apreensão do veículo Toyota Hilux, cor preta, placas PNC0E73, bem como a continuidade de sua utilização pela autoridade policial, exclusivamente para fins de diligências e atividades investigativas, até o trânsito em julgado da sentença penal condenatória; 

In [31]:
# BLOCO 6: Salvar




## Extração em massa

In [32]:
# Criar lista somente com número do processo

# SELEÇÃO E LIMPEZA DA COLUNA:
# 1. Seleciona a coluna "numeroProcesso" do DataFrame filtrado (df_sentenca).
# 2. .astype(str) garante que todos os valores na Series sejam tratados como strings.
# 3. .str.strip() remove quaisquer espaços em branco iniciais ou finais indesejados.
lista_processos = df_sentenca["numeroProcesso"].astype(str).str.strip()

# Criar DataFrame final

# CRIAÇÃO DE DATAFRAME DE CONTROLE:
# Cria um novo DataFrame (df_lista) a partir da Series 'lista_processos'.

df_lista = pd.DataFrame({"numero_processo": lista_processos})

# Salvar arquivo para ser usado no Bloco 7

# PERSISTÊNCIA DE DADOS:
# Salva o DataFrame de controle em um arquivo CSV.
# encoding="utf-8-sig": Codificação recomendada para CSVs para garantir a
# compatibilidade com programas como Excel e evitar problemas com caracteres especiais.
df_lista.to_csv("lista_processos_1848.csv", index=False, encoding="utf-8-sig")

print("Arquivo salvo: lista_processos_1848.csv")
print("Total de processos salvos:", len(df_lista))
# Verificação visual do DataFrame salvo.
df_lista.head()

Arquivo salvo: lista_processos_1848.csv
Total de processos salvos: 1848


,numero_processo
15,02033411620258060298
37,00255308720258060001
65,02190163720258060001
85,02019078920258060298
112,02928082920228060001


## Resumo Didático (Foco na Preparação do Input)
Este bloco de código é um exemplo de pré-processamento para automação em lote:

Limpeza de Tipos (.astype(str).str.strip()): É uma etapa fundamental antes de usar strings como input para URLs ou buscas. Garante que não haja números formatados de forma estranha ou espaços em branco que quebrem a URL do TJCE.

Criação de Input Persistente: Salvar a lista de números de processo em um CSV (aqui, lista_processos_708.csv) permite que você use este arquivo como o input central para um script de web scraping em lote (o futuro Bloco 7). Isso é importante para:

Continuidade: Se o Bloco 7 falhar no processo 50, você pode reiniciar do zero.

Desacoplamento: O script de scraping (Bloco 7) não precisa saber como os dados foram filtrados; ele apenas consome o arquivo CSV.

In [33]:
# ============================================================
# BLOCO 7 — AUTOMAÇÃO EM MASSA (TJCE) — VERSÃO FINAL 
# ============================================================
# -------------------------
# CONFIGURAÇÕES INICIAIS
# -------------------------
# Define o nome do arquivo CSV de entrada gerado no bloco anterior.
CSV_LISTA = "lista_processos_1848.csv"
# Define o nome da coluna que contém os números dos processos nesse CSV.
COLUNA_PROCESSO = "numero_processo"

# Índices de início e fim do lote a ser processado (útil para rodar em pedaços).
start_idx = 0      # <-- ajuste antes de rodar (Índice inicial da lista)
end_idx   = 1     # <-- ajuste antes de rodar (Índice final da lista)

# Define os diretórios de saída e log.
OUT_DIR = "movimentos_lotes"
LOG_DIR = "logs"
# Cria os diretórios se eles não existirem (exist_ok=True evita erro se já existirem).
os.makedirs(OUT_DIR, exist_ok=True)
os.makedirs(LOG_DIR, exist_ok=True)

# Função para gerar o caminho e nome do arquivo de saída do lote.
# Formato: movimentos_001_004.csv (se start_idx=0 e end_idx=3)
def arquivo_lote(i0, i1):
    # os.path.join constrói o caminho de forma segura em qualquer sistema operacional.
    return os.path.join(OUT_DIR, f"movimentos_{i0+1:03d}_{i1+1:03d}.csv")

# Define os caminhos completos para os arquivos de log.
LOG_ERR = os.path.join(LOG_DIR, "log_erros_bloco7.csv")
LOG_OK  = os.path.join(LOG_DIR, "log_sucesso_bloco7.csv")

# -------------------------
# CARREGAR LISTA DE PROCESSOS
# -------------------------
# Lê o CSV de input. dtype=str garante que o número do processo não seja interpretado como float/int.
df_lista = pd.read_csv(CSV_LISTA, dtype=str)
# Seleciona a coluna, remove NaNs, converte para string e transforma em uma lista Python.
lista_procs = df_lista[COLUNA_PROCESSO].dropna().astype(str).tolist()
N = len(lista_procs)

print(f"🔍 Total de processos na lista: {N}")

# Tratamento de segurança para os índices de lote.
if start_idx < 0: start_idx = 0
if end_idx >= N: end_idx = N - 1
if start_idx > end_idx:
    # Sai do script se os índices estiverem invertidos.
    raise SystemExit("ERRO: start_idx > end_idx")

# Extrai o subconjunto da lista (o lote a ser processado).
# O +1 garante que o end_idx (inclusivo) seja transformado corretamente para o slicing do Python (exclusivo).
sub_lista = lista_procs[start_idx:end_idx+1]
print(f"➡ Processando {len(sub_lista)} processos (índices {start_idx}..{end_idx})")

# -------------------------
# Função para abrir driver
# -------------------------
# Modularização: Encapsula toda a configuração do Selenium para ser chamada facilmente.
def abrir_driver():
    options = Options()
    # Configurações de anti-detecção e maximização (conforme Bloco 1).
    options.add_argument("--start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("--disable-notifications")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option("useAutomationExtension", False)
    
    # Gerenciamento automático do ChromeDriver.
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    # Define a espera explícita.
    wait = WebDriverWait(driver, 20)
    return driver, wait # Retorna os dois objetos essenciais para interação

# -------------------------------
# Função universal de extração de linha
# -------------------------------
# Função robusta para extrair dados de uma linha de movimentação (tr), lidando com variações de layout.
def extrair_linha_mov(tr, numero_processo):
    try:
        # 1) Tenta a extração padrão via classes CSS (Método mais limpo e rápido)
        try:
            # Tenta encontrar e limpar o texto da data.
            td_data = tr.find_element(By.CSS_SELECTOR, "td.dataMovimentacao").text.strip()
        except:
            td_data = "" # Se a data falhar, seta como vazio.

        try:
            # Tenta encontrar a descrição. get_attribute("innerText") é robusto para pegar texto interno.
            td_desc = tr.find_element(By.CSS_SELECTOR, "td.descricaoMovimentacao").get_attribute("innerText").strip()
        except:
            td_desc = ""

        # Normaliza a descrição removendo múltiplos espaços em branco.
        td_desc = " ".join(td_desc.split())

        # Se os dois campos foram encontrados (não vazios), retorna o resultado.
        if td_data and td_desc:
            return (numero_processo, td_data, td_desc)

        # 2) Fallback via qualquer <td> (Se as classes CSS falharem)
        # Busca todas as células da linha (<td>).
        tds = tr.find_elements(By.TAG_NAME, "td")
        if len(tds) >= 2:
            txt0 = tds[0].text.strip() # Pega o texto da primeira célula (espera-se ser a data).
            txtN = " ".join(tds[-1].text.split()) # Pega o texto da última célula (espera-se ser a descrição).

            # Verifica se o primeiro campo tem o formato de data MM/DD/AAAA.
            if re.match(r"\d{2}/\d{2}/\d{4}", txt0):
                return (numero_processo, txt0, txtN)

        # 3) Leitura Bruta (Último recurso: extrair tudo e usar Regex)
        # Pega todo o texto interno da linha <tr>.
        brute = tr.get_attribute("innerText").strip()
        brute = " ".join(brute.split())

        # Usa Expressão Regular (re.search) para encontrar o primeiro padrão de data.
        m = re.search(r"\d{2}/\d{2}/\d{4}", brute)
        data_bruta = m.group() if m else "" # Pega a data se encontrada.

        # A descrição é o texto bruto menos a data encontrada.
        desc_final = brute.replace(data_bruta, "").strip()

        # Se a data e a descrição final forem encontradas, retorna o resultado.
        if data_bruta and desc_final:
            return (numero_processo, data_bruta, desc_final)

        # Se todos os fallbacks falharem, retorna None.
        return None
    except:
        return None # Erro inesperado

# -------------------------------
# Função principal para extrair um processo
# -------------------------------
# Função que encapsula toda a lógica de navegação e extração para um único processo.
def extrair_movimentos_para_processo(driver, wait, numero, tentativas_max=2):

    # Função interna para quebrar o número CNJ em suas partes (conforme Bloco 2).
    def quebrar_cnj(n):
        n = n.replace(".", "").replace("-", "").zfill(20) # Limpa pontos/hífens e preenche com zeros (segurança).
        # Retorna as partes: Número Sequencial, Dígito Verificador, Ano, Foro.
        return n[:7], n[7:9], n[9:13], n[16:20]

    # Tenta extrair o processo múltiplas vezes, em caso de falha de rede/timeout.
    for tentativa in range(1, tentativas_max+1):
        try:
            # Volta para a página de consulta principal (limpa a URL anterior).
            driver.get("https://esaj.tjce.jus.br/cpopg/open.do")
            time.sleep(2)

            # Quebra o número do processo CNJ.
            seq, dv, ano, foro = quebrar_cnj(numero)

            # Preenchimento do Campo Principal (sequencial, DV, ano).
            campo_principal = wait.until(
                EC.presence_of_element_located((By.ID, "numeroDigitoAnoUnificado"))
            )
            campo_principal.clear()
            campo_principal.send_keys(f"{seq}-{dv}.{ano}")

            # Preenchimento do Campo do Foro (unidade de origem).
            campo_foro = wait.until(
                EC.presence_of_element_located((By.ID, "foroNumeroUnificado"))
            )
            campo_foro.clear()
            campo_foro.send_keys(foro)

            time.sleep(0.5)

            # CLIQUE: Espera o botão de consulta ficar clicável e clica.
            botao = wait.until(
                EC.element_to_be_clickable((By.ID, "botaoConsultarProcessos"))
            )
            botao.click()

            time.sleep(3) # Aguarda a página de resultados carregar.

            # ---- clicar no "Mais" até sumir (Lógica do Bloco 4) ----
            XPATH_MAIS = "//a[@id='linkmovimentacoes' and contains(., 'Mais')]"
            clicks = 0

            # Loop de carregamento dinâmico.
            while True:
                try:
                    # Espera curta (3s) para o botão reaparecer.
                    b = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, XPATH_MAIS))
                    )
                    # Garante que o botão esteja visível.
                    driver.execute_script("arguments[0].scrollIntoView();", b)
                    time.sleep(0.4)
                    b.click()
                    clicks += 1
                    time.sleep(1)
                    # Limite de segurança para evitar loops infinitos em processos muito longos.
                    if clicks > 50:
                        break
                except:
                    # Se o botão não for encontrado ou o clique falhar, o carregamento terminou.
                    break

            # ---- localizar tabela e extrair ----
            # Tenta localizar o tbody pelo ID (método mais rápido).
            try:
                tbody = driver.find_element(By.ID, "tabelaTodasMovimentacoes")
            except:
                tbody = None # Se falhar, tenta a busca global.

            if tbody:
                # Busca linhas DENTRO do tbody encontrado.
                linhas = tbody.find_elements(By.CSS_SELECTOR, "tr.containerMovimentacao")
            else:
                # Busca linhas em todo o documento (fallback).
                linhas = driver.find_elements(By.CSS_SELECTOR, "tr.containerMovimentacao")

            movimentos = []
            # Itera sobre as linhas (<tr>) encontradas e usa a função robusta de extração.
            for tr in linhas:
                mov = extrair_linha_mov(tr, numero)
                if mov:
                    movimentos.append(mov)

            return movimentos # Retorna a lista de movimentos se a extração for bem-sucedida.

        except Exception as e:
            # Captura a falha na navegação ou clique e tenta novamente.
            print(f"⚠ Tentativa {tentativa}/{tentativas_max} falhou para {numero}: {e}")
            time.sleep(2) # Pausa antes de tentar novamente.

    # Se o loop de tentativas terminar sem sucesso, levanta uma exceção final.
    raise Exception(f"Falha repetida ao extrair {numero}")

# -------------------------
# EXECUÇÃO
# -------------------------
# Abre o navegador e define os objetos driver e wait.
driver, wait = abrir_driver()
# Inicializa listas para coletar todos os dados extraídos e os logs.
registros = []
erros = []
sucessos = []

print("Iniciando a extração em massa...")

# Loop principal: Itera sobre a lista de processos do lote.
# start=start_idx+1 garante que a numeração da impressão seja correta.
for idx, proc in enumerate(sub_lista, start=start_idx+1):
    print(f"\n➡ ({idx}/{end_idx+1}) Processando: {proc}")
    try:
        # Chama a função de extração para o processo atual.
        movs = extrair_movimentos_para_processo(driver, wait, proc)

        if movs:
            # Processa os movimentos extraídos (lista de tuplas) para o formato de dicionário (para o DataFrame final).
            for m in movs:
                registros.append({
                    "numero_processo": m[0],
                    "data_movimento": m[1],
                    "descricao_movimento": m[2]
                })
            # Adiciona ao log de sucesso.
            sucessos.append([proc, len(movs)])
            print(f"    ✅ {len(movs)} movimentos extraídos.")
        else:
            # Adiciona ao log de erro (processo sem movimentos).
            erros.append([proc, "SEM MOVIMENTOS"])
            print("    ⚠ Nenhum movimento encontrado.")

    except Exception as e:
        # Adiciona ao log de erro (falha na função principal).
        erros.append([proc, str(e)])
        print(f"    Erro ao processar: {e}")

    time.sleep(2) # Pausa entre processos para ser educado com o servidor.

# Encerramento obrigatório do navegador e do processo do ChromeDriver.
driver.quit()

# -------------------------
# SALVAMENTO
# -------------------------
# Converte a lista final de registros em DataFrame.
df_out = pd.DataFrame(registros)
# Define o nome do arquivo de saída do lote (ex: movimentos_001_004.csv).
saida = arquivo_lote(start_idx, end_idx)
# Salva os dados extraídos.
df_out.to_csv(saida, index=False, encoding="utf-8-sig")

print(f"\n Lote salvo em: {saida} — Registros: {len(df_out)}")

# Cria DataFrames para os logs de erro e sucesso.
df_err = pd.DataFrame(erros, columns=["processo","erro"])
df_ok  = pd.DataFrame(sucessos, columns=["processo","qtd_mov"])

# Salvamento dos Logs:
# mode="a" (append) adiciona ao final do arquivo.
# header=not os.path.exists(...) garante que o cabeçalho seja escrito apenas na primeira vez.
df_err.to_csv(LOG_ERR, mode="a", index=False, header=not os.path.exists(LOG_ERR), encoding="utf-8-sig")
df_ok.to_csv(LOG_OK, mode="a", index=False, header=not os.path.exists(LOG_OK), encoding="utf-8-sig")

print(" Logs atualizados.")
print("\n BLOCO 7 FINALIZADO.")

🔍 Total de processos na lista: 1848
➡ Processando 2 processos (índices 0..1)
Iniciando a extração em massa...

➡ (1/2) Processando: 02033411620258060298
    ✅ 16 movimentos extraídos.

➡ (2/2) Processando: 00255308720258060001
    ✅ 19 movimentos extraídos.

 Lote salvo em: movimentos_lotes\movimentos_001_002.csv — Registros: 35
 Logs atualizados.

 BLOCO 7 FINALIZADO.


## Resumo Didático (Foco na Automação em Lote)
Este é o código mais importante do projeto, que transforma o script de teste em uma ferramenta de coleta de dados em massa.

Modularização (abrir_driver, extrair_movimentos_para_processo): O código está dividido em funções. Isso facilita a leitura e o reuso. A função extrair_movimentos_para_processo encapsula a lógica de navegação (limpar/preencher campos, clicar no "Mais") e o ciclo de tentativas, tornando o loop principal limpo e focado no log.

Controle de Lote (start_idx, end_idx): Permite que você processe a lista em pedaços (ex: processos 1 a 100, depois 101 a 200). Isso é vital para grandes bases de dados, pois evita que a memória do Python estoure e permite retomar o scraping após falhas.

Robustez de Extração (extrair_linha_mov): Esta função é um excelente exemplo de fallback. Ela tenta o método ideal (CSS Classes), e se falhar, tenta métodos mais brutos (posição <td>, Regex na string), garantindo que os dados sejam extraídos mesmo que o HTML da página mude ligeiramente.

Logging (LOG_ERR, LOG_OK): Salvar os resultados em dois logs separados (log_sucesso_bloco7.csv e log_erros_bloco7.csv) é uma prática profissional. Mesmo que o script falhe, você sabe exatamente quais processos foram concluídos e por que os outros falharam. O uso de mode="a" (append) garante que os logs sejam continuamente atualizados.

In [34]:
import glob

# -------------------------
# CONFIGURAÇÃO
# -------------------------
PASTA_LOTES = "movimentos_lotes"
ARQUIVO_FINAL = "dataset_movimentos_consolidado.csv"

# -------------------------
# 1) LER TODOS OS CSVs
# -------------------------
arquivos = sorted(glob.glob(os.path.join(PASTA_LOTES, "movimentos_*.csv")))

print(f" Arquivos encontrados: {len(arquivos)}")
for a in arquivos:
    print(" -", os.path.basename(a))

dfs = []

for arq in arquivos:
    df = pd.read_csv(arq, dtype=str)
    df["arquivo_origem"] = os.path.basename(arq)  # rastreabilidade
    dfs.append(df)

df_all = pd.concat(dfs, ignore_index=True)

print(f"\n Registros totais (antes de limpeza): {len(df_all)}")

# -------------------------
# 2) LIMPEZA BÁSICA
# -------------------------
# remover linhas totalmente vazias
df_all = df_all.dropna(how="all")

# remover duplicatas exatas
df_all = df_all.drop_duplicates(
    subset=["numero_processo", "data_movimento", "descricao_movimento"]
)

print(f" Registros após remover duplicatas: {len(df_all)}")

# -------------------------
# 3) NORMALIZAÇÃO DE TEXTO
# -------------------------
df_all["descricao_movimento"] = (
    df_all["descricao_movimento"]
    .astype(str)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

# -------------------------
# 4) SALVAR DATASET FINAL
# -------------------------
df_all.to_csv(ARQUIVO_FINAL, index=False, encoding="utf-8-sig")

print(f"\n Dataset consolidado salvo em: {ARQUIVO_FINAL}")

# -------------------------
# 5) VISÃO GERAL
# -------------------------
print("\n Visão geral:")
print("Processos únicos:", df_all["numero_processo"].nunique())
print("Movimentações totais:", len(df_all))


 Arquivos encontrados: 1
 - movimentos_001_002.csv

 Registros totais (antes de limpeza): 35
 Registros após remover duplicatas: 33

 Dataset consolidado salvo em: dataset_movimentos_consolidado.csv

 Visão geral:
Processos únicos: 2
Movimentações totais: 33
